In [ ]:
import difflib
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/esim-me-test-4*.resim")

for path in PATHS:
    with open(path, "rb") as f:
        recorder: OperationRecorder = pickle.load(f)
        print(f"Processing {path}")

    def get_successfully_mutated_data(tree: MutationTreeNode) -> bool:
        if tree.leaf:
            return tree.failure_reason is None

        has_successful_child = False

        for child in tree.children:
            is_successful = get_successfully_mutated_data(child)

            if is_successful:
                has_successful_child = True

            if (
                is_successful
                and tree.recording
                and tree.mutation_type != MutationType.NONE
            ):
                diff = difflib.ndiff(
                    tree.recording.original_apdu.data.hex(),
                    tree.recording.mutated_apdu.data.hex(),
                )
                pprint(
                    {
                        "recording": path,
                        "function": tree.func_name,
                        "mutation_type": tree.mutation_type,
                        "original_apdu": tree.recording.original_apdu.data.hex(),
                        "mutated_apdu": tree.recording.mutated_apdu.data.hex(),
                        "diff": "".join(diff),
                    }
                )

        return has_successful_child

    get_successfully_mutated_data(recorder.root)

Processing recordings/esim-me-test-4_ProfileDownloadScenario_DeterministicMutationEngine.resim
{'diff': '  b  f  2  0- 0- 0',
 'function': 'get_euicc_info_1',
 'mutated_apdu': 'bf20',
 'mutation_type': <MutationType.TRUNCATE: 'truncate'>,
 'original_apdu': 'bf2000',
 'recording': 'recordings/esim-me-test-4_ProfileDownloadScenario_DeterministicMutationEngine.resim'}


In [11]:
import difflib
import glob
import pickle
from pprint import pprint

from resimulate.euicc.mutation.types import MutationType
from resimulate.euicc.recorder.operation import MutationTreeNode
from resimulate.euicc.recorder.recorder import OperationRecorder

PATHS = glob.glob("recordings/5ber*.resim")

for path in PATHS:
    with open(path, "rb") as f:
        try:
            recorder: OperationRecorder = pickle.load(f)
        except Exception as e:
            continue

        print(f"Processing {path}")

    recorder.root.print_tree()

    def get_auth_errors(tree: MutationTreeNode):
        if (
            tree.mutation_type != MutationType.NONE
            and (tree.is_successful_branch() or tree.children != [])
            and tree.func_name not in ["get_euicc_info_1", "get_euicc_challenge"]
            and tree.recording.original_apdu.data.hex()
            != tree.recording.mutated_apdu.data.hex()
        ):
            diff = difflib.ndiff(
                tree.recording.original_apdu.data.hex(),
                tree.recording.mutated_apdu.data.hex(),
            )
            pprint(
                {
                    "recording": path,
                    "function": tree.func_name,
                    "mutation_type": tree.mutation_type,
                    "original_apdu": tree.recording.original_apdu.data.hex(),
                    "mutated_apdu": tree.recording.mutated_apdu.data.hex(),
                    "response": tree.recording.response_data,
                    "diff": "".join(diff),
                }
            )

        for child in tree.children:
            get_auth_errors(child)

    get_auth_errors(recorder.root)

Processing recordings/5ber-test-2_ProfileDownloadScenario_DeterministicMutationEngine.resim
 root 
   ├── get_euicc_challenge (bitflip, ApduException)
   ├── get_euicc_challenge (zero_block, ApduException)
   ├── get_euicc_challenge (truncate, ApduException)
   ├── get_euicc_challenge (random_byte, ApduException)
   ├── get_euicc_challenge (shuffle_blocks, ApduException)
   └── get_euicc_challenge (_none, success)
      ├── get_euicc_info_1 (random_byte, ApduException)
      ├── get_euicc_info_1 (bitflip, ApduException)
      ├── get_euicc_info_1 (truncate, success)
      │  ├── authenticate_server (bitflip, ApduException)
      │  ├── authenticate_server (truncate, InvalidCertificate)
      │  │  ├── prepare_download (_none, success)
      │  │  │  └── initialiseSecureChannelRequest (shuffle_blocks, ApduException)
      │  │  ├── prepare_download (truncate, InvalidCertificate)
      │  │  └── prepare_download (bitflip, ApduException)
      │  ├── authenticate_server (_none, success)
 

In [12]:
from resimulate.asn import asn
from pprint import pprint

response = "bf3882067ba08206773081f98008000000001959b7be83107273702e74727570686f6e652e636f6d8410c8a65776accdb4f8638143b65570a05cbf2281a6810302030182030202028303040200840d810100820400075944830237338505077f3e1f808603090200870302030088020490a92c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfbaa2c04140b1359633d2ab469761ca9d82b2229350db722d7041481370f5125d0b1d408d4c3b232e6d25e795bebfb8b010004030100000c0d45442d5a492d55502d30383234a01f801351522d472d35432d314c532d3157315a395037a108800435290611a1005f37407a41016bbfa6ec924a1ce51175b1647f6c1719e5b1c45ccf5af9a25562e7e0ea04dc2a45d95a746de6018bad0e89f84ea43819cee6b2d491afc948f39072d77630820239308201e0a00302010202110089086030202200000024000086963302300a06082a8648ce3d0403023079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693020170d3234303832333133353234395a180f39393939313233313233353935395a305531283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c7464312930270603550405132038393038363033303230323230303030303032343030303038363936333330333059301306072a8648ce3d020106082a8648ce3d03010703420004f3e8d74899a2416bde865bbd4b7e02d2842d2d28deaaa8820d375ccd2b36bea7779897f2bf9946966b1d58110d15a56e1aaea75872eb6b1d242cec37e8a7eba0a36b3069301f0603551d230418301680143a6b2ca4585d9c95c5947abdd3bb1b0169acef72301d0603551d0e04160414499ab8cda957ba7db854023c6500f75e3b8cb549300e0603551d0f0101ff04040302078030170603551d200101ff040d300b3009060767811201020101300a06082a8648ce3d040302034700304402203a38d9434e4110d6ccb312bf130f8c4670e2601b2cab00d9e1c5046893f07b92022033ec66a2e3af013daff426da43a5efc09b9495267338972476f0d1d0acdfc5ab308202f73082029ea00302010202105986939cf376fdd1013e8f1529307748300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3139313132313030303030305a170d3439313132303233353935395a3079310b300906035504061302434e31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c746431153013060355040b130c45617374636f6d7065616365312930270603550403132045617374636f6d70656163652e45554d2e436f6e73756d65722e5a68756861693059301306072a8648ce3d020106082a8648ce3d030107034200042fb0e6c5292e90321b0f1c69e88d5c3cf8c0e5f79e5c6f2c588552a74ab894aa0b4414b349f4616cdde9629dcaf955b7c39712eeaf46101af686d44f8b2dfc07a382013b30820137301d0603551d0e041604143a6b2ca4585d9c95c5947abdd3bb1b0169acef7230120603551d130101ff040830060101ff02010030170603551d200101ff040d300b3009060767811201020102304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302010630510603551d1e0101ff04473045a0433041a43f303d31283026060355040a131f45617374636f6d706561636520546563686e6f6c6f677920436f2e2c4c74643111300f06035504051308383930383630333030160603551d11040f300d880b2b06010401838a1d010202301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d040302034700304402203c5fd987d7cabb5172489647a8563016be05de44b06ce285a2d6790bcdf408ca02202272886d929fc9434153068c8be3521bc371e9101a18a56e9baa7dd936592a1e"
data = asn.decode(
    "AuthenticateServerResponse",
    bytes.fromhex(response),
    check_constraints=True,
)
pprint(data)


('authenticateResponseOk',
 {'euiccCertificate': {'signature': (bytearray(b'0D\x02 :8\xd9CNA\x10\xd6'
                                               b'\xcc\xb3\x12\xbf\x13\x0f\x8cF'
                                               b'p\xe2`\x1b,\xab\x00\xd9'
                                               b'\xe1\xc5\x04h\x93\xf0{\x92'
                                               b'\x02 3\xecf\xa2\xe3\xaf'
                                               b'\x01=\xaf\xf4&\xdaC\xa5'
                                               b'\xef\xc0\x9b\x94\x95&s8'
                                               b'\x97$v\xf0\xd1\xd0\xac\xdf'
                                               b'\xc5\xab'),
                                     560),
                       'signatureAlgorithm': {'algorithm': '1.2.840.10045.4.3.2'},
                       'tbsCertificate': {'extensions': [{'critical': False,
                                                          'extnID': '2.5.29.35',
                       

In [1]:
import difflib
import json

from IPython.display import HTML, display
from pprint import pprint

from resimulate.asn import asn

# authenticate_server_mutated = "bf3882027130408008000000001954e67b8110a42accee098234fe163aafb0837d46bf83107273702e74727570686f6e652e636f6d841071b05cf7c9c63911ea9bd771a611ca915f3740cc35a76b34ca3e5755edf9d7f667f2130c9580c96a52e8d367a4c729dba65790754547ef36c7a2e7f4186a9b8ebed3d80f9275a0b744b379d1e782bbe7ac41df041481370f5125d0b1d408d4c3b232e6d25e795bebfb308201d2308201cea003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca3817930817630170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300c0603551d0f0101ff04020302"
authenticate_server_mutated = "bf3882027130408008000000001959b7be81108f838285224dc7a4233fce0b97931a8583107273702e74727570686f6e652e636f6d8410c8a65776accdb4f8638143b65570a05c5f37402c519f80b3c6f0cec9c3c582af5974265cf09a67cd5b6ebee7205555f6129899ae02ba0ebad7aed89be4127d6e8e41e768211fc304465d23267ebca0a1a11f7a041481370f5125d0b1d408d4c3b232e6d25e795bebfb308201d2308201cea003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca3817930817630170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300c0603551d0f0101ff04020302"
authenticate_server_original = "bf3882034330408008000000001954e5df81106ab410e0d56d874bc95f478e65d6664683107273702e74727570686f6e652e636f6d8410a6bdc830e18bc5352170dc6eddb2199b5f3740fdaf7e7d13614cc7cf2f8b12c6d0cc8b7ca0fcceb2d870445cf84329964d16cb3b470e5de3f7bb984dfef55d28571cc706268e7c89e7d7d0265021a11bfbb120041481370f5125d0b1d408d4c3b232e6d25e795bebfb3082028330820228a003020102021064055185218bd277dfa47d746d994324300a06082a8648ce3d040302304431183016060355040a130f47534d204173736f63696174696f6e312830260603550403131f47534d204173736f63696174696f6e202d205253503220526f6f7420434931301e170d3234303232373030303030305a170d3237303232363233353935395a306c310b3009060355040613024742310f300d060355040713064c6f6e646f6e31253023060355040a131c545020476c6f62616c204f7065726174696f6e73204c696d69746564310c300a060355040b1303525350311730150603550403130e31474c4f42414c20534d2d44502b3059301306072a8648ce3d020106082a8648ce3d030107034200046fbf48592158490c37d65380f6cf1c15ec02fb0e04cb9788a371731bd2afa627ea906c12bd02b74a05c08163ce1effdc357332dcd9c3139729922221d6a7a51ca381d33081d030170603551d200101ff040d300b3009060767811201020104304d0603551d1f044630443042a040a03e863c687474703a2f2f67736d612d63726c2e73796d617574682e636f6d2f6f66666c696e6563612f67736d612d727370322d726f6f742d6369312e63726c300e0603551d0f0101ff04040302078030160603551d11040f300d880b2b0601040181ec45010101301d0603551d0e0416041418f04c54d7a617353b0662e1bd055474d19cd33f301f0603551d2304183016801481370f5125d0b1d408d4c3b232e6d25e795bebfb300a06082a8648ce3d0403020349003046022100bfbfd9279495d8a09531e55c9728c99b7b7533629fb3d564ac0c5cad09405972022100f4baf4758e94d403a6f66eb746a3c6d2fbf24e19067128182e7794fad1fa1fbaa01f801351522d472d35432d314c532d3157315a395037a108800435290611a100"

mutated = asn.decode(
    "AuthenticateServerRequest", data=bytes.fromhex(authenticate_server_mutated)
)
original = asn.decode(
    "AuthenticateServerRequest", data=bytes.fromhex(authenticate_server_original)
)

pprint(original)
pprint(mutated)

diff = difflib.HtmlDiff().make_file(
    mutated,
    original,
    fromdesc="Mutated",
    todesc="Original",
    context=True,
    numlines=1,
)

display(HTML(diff))


{'ctxParams1': ('ctxParamsForCommonAuthentication',
                {'deviceInfo': {'deviceCapabilities': {}, 'tac': b'5)\x06\x11'},
                 'matchingId': 'QR-G-5C-1LS-1W1Z9P7',
                 'operationType': (b'\x80', 1)}),
 'euiccCiPKIdToBeUsed': b'\x817\x0fQ%\xd0\xb1\xd4\x08\xd4\xc3\xb22\xe6\xd2^'
                        b'y[\xeb\xfb',
 'serverCertificate': {'signature': (bytearray(b'0F\x02!\x00\xbf\xbf\xd9'
                                               b"'\x94\x95\xd8\xa0\x951\xe5"
                                               b'\\\x97(\xc9\x9b{u3'
                                               b'b\x9f\xb3\xd5d\xac\x0c\\'
                                               b'\xad\t@Yr\x02!\x00'
                                               b'\xf4\xba\xf4u\x8e\x94\xd4\x03'
                                               b'\xa6\xf6n\xb7F\xa3\xc6\xd2'
                                               b'\xfb\xf2N\x19\x06q(\x18'
                                               b'.w\

In [ ]:
from copy import deepcopy
import logging

from resimulate.asn import asn
from resimulate.euicc.card import Card
from resimulate.euicc.exceptions import (
    ApduException,
    AuthenticateException,
    NothingToDeleteError,
)
from resimulate.euicc.models.activation_profile import ActivationProfile
from resimulate.euicc.models.reset_option import ResetOption
from resimulate.euicc.mutation.deterministic_engine import DeterministicMutationEngine
from resimulate.euicc.transport.apdu import APDUPacket
from resimulate.euicc.transport.pcsc_link import PcscLink
from resimulate.smdp.client import SmdpClient
from resimulate.euicc.mutation.types import MutationType
from resimulate.smdp.models import InitiateAuthenticationResponse

from pySim.utils import sw_match

from osmocom.utils import h2b


truncation_point = -327


def store_data(
    card: Card,
    request_type: str | None = None,
    response_type: str | None = None,
    request_data: dict | None = None,
    caller_func_name: str | None = None,
    do_not_mutate: bool = False,
) -> dict | tuple | str | None:
    if request_data is None:
        request_data = dict()

    command_encoded = request_data
    if request_type is not None:
        logging.debug(f"Sending request: {request_type}")
        command_encoded = asn.encode(
            request_type,
            request_data,
            # check_constraints=True,
        )

    if len(command_encoded) > 65536:
        raise ValueError("Data too long")

    mutated_data = command_encoded[:truncation_point]
    apdu = APDUPacket(cla=0x80, ins=0xE2, p1=0x91, p2=0x00, data=mutated_data)
    data, sw = card.isd_r.link.send_apdu_with_mutation(
        "authenticate_server", apdu, do_not_mutate=do_not_mutate
    )
    print(f"Data send: {len(mutated_data)} bytes, received: {len(data)} bytes")

    if not any([sw_match(sw, pattern) for pattern in ["9000", "61??"]]):
        raise ApduException(sw)

    if not data:
        return None

    if response_type is None:
        return data

    return asn.decode(response_type, h2b(data), check_constraints=False)


def authenticate_server(
    card: Card,
    authentication: InitiateAuthenticationResponse,
    matching_id: str,
    imei: str | None = None,
) -> dict:
    device_info = {"tac": bytes.fromhex("35290611"), "deviceCapabilities": dict()}
    if imei:
        device_info["imei"] = imei

    request = {
        "serverSigned1": authentication.server_signed_1,
        "serverSignature1": authentication.server_signature_1,
        "euiccCiPKIdToBeUsed": authentication.euicc_ci_pki_to_be_used,
        "serverCertificate": authentication.server_certificate,
        "ctxParams1": (
            "ctxParamsForCommonAuthentication",
            {
                "matchingId": matching_id,
                "deviceInfo": device_info,
            },
        ),
    }

    logging.debug(f"AuthenticateServerRequest: {request}")

    data = store_data(
        card=card,
        request_type="AuthenticateServerRequest",
        request_data=request,
    )

    # response = asn.decode(
    #     "AuthenticateServerResponse",
    #     bytes.fromhex(data),
    #     check_constraints=False,
    # )

    # print(f"AuthenticateServerResponse: {response}")

    # result, error = response
    # if result == "authenticateResponseError":
    #     code = error.get("authenticateErrorCode")
    #     raise AuthenticateException.raise_from_code(code)

    return data


def clear_card(card: Card):
    notifications = card.isd_r.retrieve_notification_list()
    if notifications:
        card.isd_r.process_notifications(notifications)

    try:
        card.isd_r.reset_euicc_memory(reset_options=ResetOption.all())
    except NothingToDeleteError:
        pass


with PcscLink(apdu_data_size=255) as link:
    card = Card(link)

    profile = ActivationProfile.from_activation_code(
        "LPA:1$rsp.truphone.com$QR-G-5C-1LS-1W1Z9P7"  # "LPA:1$smdpp.test.rsp.sysmocom.de$TS48v5_SAIP2.1B_NoBERTLV"
    )
    while True:
        truncation_point -= 1
        print(f"Truncation point: {truncation_point}")
        clear_card(card)
        try:
            smdp_address = profile.smdp_address
            if not smdp_address:
                smdp_address = card.isd_r.get_configured_data().default_dp_address

            assert smdp_address, (
                "No SMDP address found in profile or card configuration."
            )

            smdp_client = SmdpClient(smdp_address=smdp_address, verify_ssl=False)

            logging.debug(f"Downloading profile from {smdp_address}")
            euicc_challenge = card.isd_r.get_euicc_challenge()
            euicc_info_1 = card.isd_r.get_euicc_info_1()

            init_auth = smdp_client.initiate_authentication(
                euicc_challenge, euicc_info_1
            )
            transaction_id = init_auth.transaction_id

            auth_server = authenticate_server(card, init_auth, profile.matching_id)
            authenticate_client = smdp_client.authenticate_client(
                transaction_id, auth_server
            )
            prepare_download = card.isd_r.prepare_download(
                authenticate_client, profile.confirmation_code
            )

            get_bpp_response = smdp_client.get_bound_profile_package(
                transaction_id,
                prepare_download,
            )
            notification = card.isd_r.load_bound_profile_package(get_bpp_response)
            print(notification)
        except Exception as e:
            logging.error(f"Error during profile download: {e}")
            continue

        print("SUCCESS!!!!!!!")
        print("Succeeded with Truncation point:", truncation_point)
        break

Truncation point: -328


ERROR:root:Error during profile download: Error in SMDP response: {'subjectCode': '1.6', 'reasonCode': '4.2', 'subjectIdentifier': 'Function', 'message': 'Execution Error'}


Data send: 512 bytes, received: 36 bytes
Truncation point: -329


ERROR:root:Error during profile download: Error in SMDP response: {'subjectCode': '1.6', 'reasonCode': '4.2', 'subjectIdentifier': 'Function', 'message': 'Execution Error'}


Data send: 511 bytes, received: 36 bytes
Truncation point: -330


...

Data send: 506 bytes, received: 36 bytes
Truncation point: -335


ERROR:root:Error during profile download: Error in SMDP response: {'subjectCode': '1.6', 'reasonCode': '4.2', 'subjectIdentifier': 'Function', 'message': 'Execution Error'}


Data send: 505 bytes, received: 36 bytes
Truncation point: -336


ERROR:root:Error during profile download: Error in SMDP response: {'subjectCode': '1.6', 'reasonCode': '4.2', 'subjectIdentifier': 'Function', 'message': 'Execution Error'}


Data send: 504 bytes, received: 36 bytes
Truncation point: -337


In [11]:
from resimulate.euicc.mutation.types import MutationType

sorted(set(MutationType).difference(MutationType.TRUNCATE), reverse=True).pop()

<MutationType.NONE: '_none'>